In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.basketball-reference.com'

r = requests.get(url)

soup = BeautifulSoup(r.content, 'lxml') # object that represents the html file

In [3]:
import re

teams_parsed = soup.find_all('select', {'id': 'select_team'})

list_all_teams = str(teams_parsed[0]).split('\n')[2:-1]

team_list = []
for team in list_all_teams:
    match = re.search(r'"(.*?)"', team)
    output = match.group()[1:-1]
    team_list.append(output)
    

In [4]:
team_list

['/teams/ATL',
 '/teams/BOS',
 '/teams/BRK',
 '/teams/CHI',
 '/teams/CHO',
 '/teams/CLE',
 '/teams/DAL',
 '/teams/DEN',
 '/teams/DET',
 '/teams/GSW',
 '/teams/HOU',
 '/teams/IND',
 '/teams/LAC',
 '/teams/LAL',
 '/teams/MEM',
 '/teams/MIA',
 '/teams/MIL',
 '/teams/MIN',
 '/teams/NOP',
 '/teams/NYK',
 '/teams/OKC',
 '/teams/ORL',
 '/teams/PHI',
 '/teams/PHO',
 '/teams/POR',
 '/teams/SAC',
 '/teams/SAS',
 '/teams/TOR',
 '/teams/UTA',
 '/teams/WAS']

In [5]:
def get_stats(year):
    nba_info = []
    for i in team_list:
        team_url = (f'https://www.basketball-reference.com{i}/{str(year)}.html')
        team_res = requests.get(team_url)
        team_soup = BeautifulSoup(team_res.content, 'lxml')
        per_game = soup.find(name = 'table', attrs={'id': 'per_game_stats'})
        for row in per_game.find_all('tr')[1:-1]:
            player = {}
            player['Name'] = row.find('td', {'data-stat': 'name_display'}).text
            team = i[-3:]
            player['Team'] = team
            player['Age'] = row.find('td', {'data-stat' : 'age'}).text
            player['Min PG'] = row.find('td', {'data-stat' : 'mp_per_g'}).text
            player['Field Goal %'] = row.find('td', {'data-stat' : 'fg_pct'}).text
            player['Rebounds PG'] = row.find('td', {'data-stat' : 'trb_per_g'}).text
            player['Assists PG'] = row.find('td', {'data-stat' : 'ast_per_g'}).text
            player['Steals PG'] = row.find('td', {'data-stat' : 'stl_per_g'}).text
            player['Blocks PG'] = row.find('td', {'data-stat' : 'blk_per_g'}).text
            player['Turnovers PG'] = row.find('td', {'data-stat' : 'tov_per_g'}).text
            player['Points PG'] = row.find('td', {'data-stat' : 'pts_per_g'}).text

            name_td = row.find('td', {'data-stat': 'name_display'})
            a_tag = name_td.find('a')['href']
            player_url = 'https://www.basketball-reference.com' + a_tag
            player_request = requests.get(player_url)
            player_soup = BeautifulSoup(player_request.content, 'lxml')
            player_info = player_soup.find(name = 'div', attrs = {'id' : 'info', 'class' : 'players'})

            player_links = []
            for link in player_info.find_all('a'):
                player_links.append(link.get('href'))
            if 'instagram' in player_links[0]:
                player['Insta Handle'] = player_links[0].replace('https://instagram.com/', '')
            else:
                player['Insta Handle'] = 'N/A'
            
            position_tag = player_info.find('strong', string=lambda text: text and 'Position' in text)
            position = ' '.join(position_tag.next_sibling.text.strip().split()[:-1])
            player['Position'] = position
            p_tag = position_tag.find_parent('p').find_next_sibling()
            spans = p_tag.find_all('span')
            height, weight = spans[0].text, spans[1].text
            player['Height'] = height
            player['Weight (lbs)'] = weight[:-2]

            nba_info.append(player)
        
    nba_info_df = pd.DataFrame(nba_info)
    return nba_info_df
            



In [6]:
all_nba_players_2021 = get_stats(2025)
all_nba_players_2021

AttributeError: 'NoneType' object has no attribute 'find_all'